In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
attributes = {'Goalkeeping': 'keepersadv',
              'Shooting': 'shooting',
              'Passing': 'passing',
              'Pass types': 'pass_types',
              'Goal and Shot Creation': 'gca',
              'Defensive Actions': 'defense',
              'Possession': 'possession',
              'Playing Time': 'playingtime',
              'Miscellaneous Stats': 'misc'
             }

In [1]:
Leagues = { 'Serie A': ['https://fbref.com/en/comps/11/2022-2023/', '/2022-2023-Serie-A-Stats'],
            'Premier League': ['https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats'],
            'La Liga': ['https://fbref.com/en/comps/12/2022-2023/','/2022-2023-La-Liga-Stats'],
            'Bundesliga': ['https://fbref.com/en/comps/20/2022-2023/','/2022-2023-Bundesliga-Stats'],
            'Ligue 1': ['https://fbref.com/en/comps/13/2022-2023/','/2022-2023-Ligue-1-Stats']}

In [4]:
for j in Leagues.keys():
    print(j)
    for i in attributes:
        print(Leagues[j][0] + attributes[i] + Leagues[j][1])
    print()

Serie A
https://fbref.com/en/comps/11/2022-2023/keepersadv/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/shooting/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/passing/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/pass_types/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/gca/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/defense/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/possession/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/playingtime/2022-2023-Serie-A-Stats
https://fbref.com/en/comps/11/2022-2023/misc/2022-2023-Serie-A-Stats

Premier League
https://fbref.com/en/comps/9/2022-2023/keepersadv/2022-2023-Premier-League-Stats
https://fbref.com/en/comps/9/2022-2023/shooting/2022-2023-Premier-League-Stats
https://fbref.com/en/comps/9/2022-2023/passing/2022-2023-Premier-League-Stats
https://fbref.com/en/comps/9/2022-2023/pass_types/2022-2023-Premier-League-Stats
ht

In [ ]:
url = 'https://fbref.com/en/squads/dc56fe14/2023-2024/all_comps/Milan-Stats-All-Competitions#all_stats_defense'
response = requests.get(url)

In [77]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Select the table containing player statistics
    table = soup.select('table#stats_standard_combined tbody')
    
    # Check if the table is found
    if table:
        # Convert the table to string and create a new BeautifulSoup object
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find all table rows within the table
        player_rows = table_soup.find_all('tr')
        
        # Initialize an empty list to store player dictionaries
        player_data = []
        
        # Iterate over each row and extract player information
        for row in player_rows:
            # Extract player name
            player_name = row.find('th', {'data-stat': 'player'}).text.strip()
            
            # Extract other statistics
            nationality = row.find('td', {'data-stat': 'nationality'}).text.strip()
            position = row.find('td', {'data-stat': 'position'}).text.strip()
            age = row.find('td', {'data-stat': 'age'}).text.strip()
            games = row.find('td', {'data-stat': 'games'}).text.strip()
            goals = row.find('td', {'data-stat': 'goals'}).text.strip()
            assists = row.find('td', {'data-stat': 'assists'}).text.strip()
            
            # Create a dictionary for the player's statistics
            player_stats = {
                'Player': player_name,
                'Nationality': nationality,
                'Position': position,
                'Age': age,
                'Games': games,
                'Goals': goals,
                'Assists': assists
            }
            
            # Append the player's statistics to the list
            player_data.append(player_stats)
        
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(player_data)
        
        # Print the DataFrame
        print(df)
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)

                 Player Nationality Position     Age Games Goals Assists
0     Tijjani Reijnders      nl NED       MF  25-195    31     2       2
1        Theo Hernández      fr FRA       DF  26-126    28     3       6
2          Mike Maignan      fr FRA       GK  28-221    27     0       1
3       Davide Calabria      it ITA       DF  27-065    28     0       4
4     Christian Pulisic      us USA       FW  25-144    29     7       6
5           Rafael Leão      pt POR       FW  24-244    27     6       6
6        Olivier Giroud      fr FRA       FW  37-132    27    12      10
7         Fikayo Tomori     eng ENG       DF  26-052    22     3       0
8    Ruben Loftus-Cheek     eng ENG       MF  28-017    24     5       1
9           Yunus Musah      us USA       MF  21-072    24     0       1
10         Malick Thiaw      de GER       DF  22-185    16     0       0
11  Alessandro Florenzi      it ITA       DF  32-335    25     0       2
12          Yacine Adli      fr FRA       MF  23-19

In [86]:
# Example age strings
age_strings = ["25-195", "26-126"]

# Function to remove the "days" part from the age string
def clean_age(age_string):
    # Split the age string into age and days parts
    age, days = age_string.split("-")
    
    # Return only the age part
    return age

# Clean age for each example age string
cleaned_ages = [clean_age(age_str) for age_str in age_strings]
print(cleaned_ages)  # Output: ['25', '26']


['25', '26']


In [40]:
import pandas as pd

# Assuming df is your DataFrame
# For demonstration purposes, let's create a sample DataFrame
data = {'Name': ['John', 'Emma', 'Ryan'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Paris', 'London']}
df = pd.DataFrame(data)

# Export the DataFrame to a CSV file
df.to_csv('output.csv', index=False)
